#### For Colab

In [ ]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [ ]:
%%capture

requirements_dir = (drive_dir + 'transformers/examples/seq2seq/').replace(' ', '\ ')
requirements_file = requirements_dir + 'requirements.txt'
!cd $requirements_dir; pip install -r $requirements_file

!pip install transformers==4.1.1
!pip install -U pyarrow
!pip install -U wandb

## **Fine-tuning**

In [ ]:
finetune_script = '"'+drive_dir+'transformers/examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval.py"'

### **Config**

In [ ]:
import sys
sys.path.insert(0, drive_dir)
import config

import torch
torch.manual_seed = config.SEED

#import wandb
#wandb.login()

### XSUM

In [ ]:
model_name_or_path = 'facebook/bart-large'

In [ ]:
data_dir = '"'+drive_dir+'datasets/xsum"'

output_dir = '"'+drive_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_xsum"'

log_dir = drive_dir + '/logs'

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_xsum.txt"'
score_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_xsum.json"'

In [ ]:
!python3 $eval_script \
'facebook/bart-large' \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--fp16 \
--bs 16 \
--dump-args

2021-01-29 11:20:58.692415: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 1.52k/1.52k [00:00<00:00, 1.56MB/s]
Downloading: 100% 1.02G/1.02G [00:11<00:00, 90.8MB/s]
Downloading: 100% 899k/899k [00:00<00:00, 2.79MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 1.39MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 3.55MB/s]
100% 709/709 [2:46:47<00:00, 14.12s/it]
{'rouge1': 14.3257, 'rouge2': 2.5636, 'rougeL': 9.8846, 'rougeLsum': 10.2215, 'n_obs': 11333, 'runtime': 10007, 'seconds_per_sample': 0.883}


In [ ]:
!python3 $eval_script \
'facebook/bart-large-xsum' \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--fp16 \
--bs 16 \
--dump-args

2021-01-22 08:42:59.045344: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 1.36k/1.36k [00:00<00:00, 1.47MB/s]
Downloading: 100% 1.63G/1.63G [01:12<00:00, 22.5MB/s]
Downloading: 100% 899k/899k [00:00<00:00, 2.28MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 1.38MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 3.52MB/s]
100% 709/709 [1:13:00<00:00,  6.18s/it]
{'rouge1': 45.3727, 'rouge2': 22.32, 'rougeL': 37.1891, 'rougeLsum': 37.1841, 'n_obs': 11333, 'runtime': 4380, 'seconds_per_sample': 0.3865}


In [ ]:
'''
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained('facebook/bart-large')
model_config.activation_dropout = 0
model_config.attention_dropout = 0
model_config.classif_dropout = 0
model_config.min_length = 11
model_config.max_length = 63
model_config.no_repeat_ngram_size = 3
model_config.num_beams = 6
model_config.task_specific_params['summarization']['length_penalty'] = 1
model_config.task_specific_params['summarization']['min_length'] = 11
model_config.task_specific_params['summarization']['max_length'] = 62
model_config.task_specific_params['summarization']['num_beams'] = 6

model_config_dir = '"'+drive_dir+'fine-tuning/'+\
    'facebook/bart-large'.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])
'''

'\nfrom transformers import AutoConfig\nmodel_config = AutoConfig.from_pretrained(\'facebook/bart-large\')\nmodel_config.activation_dropout = 0\nmodel_config.attention_dropout = 0\nmodel_config.classif_dropout = 0\nmodel_config.min_length = 11\nmodel_config.max_length = 63\nmodel_config.no_repeat_ngram_size = 3\nmodel_config.num_beams = 6\nmodel_config.task_specific_params[\'summarization\'][\'length_penalty\'] = 1\nmodel_config.task_specific_params[\'summarization\'][\'min_length\'] = 11\nmodel_config.task_specific_params[\'summarization\'][\'max_length\'] = 62\nmodel_config.task_specific_params[\'summarization\'][\'num_beams\'] = 6\n\nmodel_config_dir = \'"\'+drive_dir+\'fine-tuning/\'+    \'facebook/bart-large\'.replace(\'/\', \'?\')+\'_config"\'\nmodel_config.save_pretrained(model_config_dir[1:-1])\n'

##### Fine tune

In [ ]:
model_name_or_path = '"' + drive_dir + 'fine-tuning/facebook?bart-large_xsum/checkpoint-4500"'
model_name_or_path

'"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/facebook?bart-large_xsum/checkpoint-4500"'

In [ ]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained('facebook/bart-large')
tok.save_pretrained(model_name_or_path[1:-1])

('/content/drive/My Drive/MAGMA: Summarization/fine-tuning/facebook?bart-large_xsum/checkpoint-4500/tokenizer_config.json',
 '/content/drive/My Drive/MAGMA: Summarization/fine-tuning/facebook?bart-large_xsum/checkpoint-4500/special_tokens_map.json',
 '/content/drive/My Drive/MAGMA: Summarization/fine-tuning/facebook?bart-large_xsum/checkpoint-4500/vocab.json',
 '/content/drive/My Drive/MAGMA: Summarization/fine-tuning/facebook?bart-large_xsum/checkpoint-4500/merges.txt',
 '/content/drive/My Drive/MAGMA: Summarization/fine-tuning/facebook?bart-large_xsum/checkpoint-4500/added_tokens.json')

In [ ]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_name_or_path \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--fp16 --warmup_steps 50 \
--attention_dropout 0 --dropout 0.1 \
--learning_rate 1.2e-4 \
--sortish_sampler --freeze_embeds \
--task summarization \
--max_source_length 1024 \
--max_target_length 60 \
--val_max_target_length 60 \
--do_train \
--max_steps 5000 \
--n_val 500 \
--logging_steps 10 --save_steps 500 --save_total_limit 100 \
--per_device_train_batch_size 1 --per_device_eval_batch_size 1 \
--gradient_accumulation_steps 128 --eval_accumulation_steps 16 \
--do_eval --evaluation_strategy steps --eval_steps 500 \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

2021-01-22 13:38:33.212043: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
01/22/2021 13:38:37 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
01/22/2021 13:38:37 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/content/drive/My Drive/MAGMA: Summarization/fine-tuning/facebook?bart-large_xsum', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=128, eval_accumulation_steps=16, learning_rate=0.00012, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=5000, war

##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

In [ ]:
!python3 $eval_script \
$model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--fp16 \
--bs 16 \
--dump-args

2021-01-25 22:37:25.067143: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
100% 709/709 [1:24:30<00:00,  7.15s/it]
{'rouge1': 44.9102, 'rouge2': 21.636, 'rougeL': 36.2378, 'rougeLsum': 36.2318, 'n_obs': 11333, 'runtime': 5070, 'seconds_per_sample': 0.4474}
